In [1]:
import numpy as np 
import os
import fastkml
import pandas as pd 
dir_move = "../gps/data/"

In [5]:
kmls = list(np.sort([kml for kml in os.listdir(dir_move) if "kml" in kml]))

In [77]:
def get_coordinates(path_kml):
    '''
    extract lat, lon, altitude info from the kml data
    '''
    doc = open(path_kml, 'rb+').read()
    k = fastkml.KML()
    k.from_string(doc)
    kmlstr = k.to_string()
    coordstr = kmlstr.split("<coordinates>")[1].split("</coordinates>")[0]
    coordlst = coordstr.split(" ")
    l = []
    for point in coordlst:
        pointlst = point.split(",")
        l.append(pointlst)
    df = pd.DataFrame(l,columns = ["lon","lat","altitude"])
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return(df)

def get_rolling_average(vec, k=5):
    rvec = np.zeros(len(vec))
    for t in range(len(vec)):
        start = np.max([t - k,0])
        end   = np.min([t + k,len(vec)]) 
        rvec[t] = np.mean(vec[start:end])
    return(rvec)

from math import sin, cos, sqrt, atan2, radians

def get_distance_btw2pts(lat1,lon1,lat2,lon2):
    '''
    approximate radius of earth in km
    https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude 
    '''

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    R = 6373.0
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return(distance)

def get_distance(df_phone):
    count = 1
    df_phone["dist"] = np.NaN
    for lat1, lat2, lon1, lon2 in zip(df_phone["lat"][:-1],df_phone["lat"][1:],
                                      df_phone["lon"][:-1],df_phone["lon"][1:]):
        dis = get_distance_btw2pts(lat1,lon1,lat2,lon2)
        df_phone["dist"].iloc[count] = dis
        count += 1
    print("  distance = {:3.1f}km".format(np.sum(df_phone["dist"])))
    return(df_phone)

def get_alt_change(alt):
    diff_alt = alt[1:] - alt[:-1]
    gain = np.sum(diff_alt[diff_alt > 0])
    loss = np.sum(diff_alt[diff_alt < 0])
    print("  Elevation gain {:3.0f}m, Elevation loss {:3.0f}m".format(gain,loss))
    print("  N samples = {:3d}".format(len(alt)))
    return gain, loss

def remove_unrealistic_alt(df):
    '''
    if the altitude change more than 20m within 10 sec, we remove such point
    '''
    shape = df.shape

    pick = np.abs(df["altitude"].iloc[1:].values - df["altitude"].iloc[:-1].values) < 20
    pick1 = np.array([True] + list(pick))
    pick2 = np.array(list(pick) + [True])
    print(df.shape)
    df = df.loc[pick1 & pick2,:]
    print(df.shape)
    df = df.loc[df["altitude"] > 0,:] 
    print("remove unrealistic altitude change shape={} --> {}".format(shape,df.shape))
    return(df)

In [79]:
fnm = "Move_2018_04_24_07_10_00_Mountaineering_Lukla_Phakding.kml"
kmls

['Move_2018_04_24_07_10_00_Mountaineering_Lukla_Phakding.kml',
 'Move_2018_04_25_08_12_00_Mountaineering_Phakding_NamcheBzaar.kml',
 'Move_2018_04_26_07_58_23_Trekking_NamcheBzaar_NamcheBzaar.kml',
 'Move_2018_04_27_07_34_33_Trekking_NamcheBzaar_Tingboche.kml',
 'Move_2018_04_28_07_39_51_Trekking_Tingboche_Dingboche.kml',
 'Move_2018_04_29_07_55_47_Trekking_Dingboche_Dingboche.kml',
 'Move_2018_04_30_08_32_42_Mountaineering_Dingboche_Lobche_part1.kml',
 'Move_2018_04_30_12_42_05_Mountaineering_Dingboche_Lobche_part2.kml',
 'Move_2018_04_30_15_38_30_Mountaineering_Dingboche_Lobche_part3.kml',
 'Move_2018_05_01_06_51_39_Mountaineering_Lobche_Gorekshep_EBC.kml',
 'Move_2018_05_02_04_20_41_Mountaineering_Gorekshep_Kalapathar_Zongla.kml',
 'Move_2018_05_03_05_41_48_Mountaineering_Zongla_ChoLa_part1.kml',
 'Move_2018_05_03_08_56_55_Mountaineering_ChoLa_Thangna_Gokyo_part2.kml',
 'Move_2018_05_04_06_40_38_Mountaineering_Gokyo_GokyoRi_part1.kml',
 'Move_2018_05_04_12_16_31_Mountaineering_Gokyo

In [85]:
dfs = []
for fnm in kmls:
    print("____________")
    print(fnm)
    df = get_coordinates(dir_move + fnm)

    ## clean the data for trekking recorded using phone 
    if "Trekking" in fnm:
        for col in df.columns:
            df[col] = get_rolling_average(df[col].values, k=5)
            
    ## remove the unrealistic values
    df = remove_unrealistic_alt(df)
    
    ## add info from calenders
    fnm = fnm.split("_")
    df["date"]  = fnm[1] + "-" + fnm[2] + "-" + fnm[3]
    df["move"]  = fnm[7]
    
    ## calculate distance between two points
    df = get_distance(df)
    
    gain, loss = get_alt_change(df["altitude"].values)
    df["gain"] , df["loss"] = gain, loss
    dfs.append(df)
    
dfs = pd.concat(dfs)

No geometries found
Object does not have a geometry


____________
Move_2018_04_24_07_10_00_Mountaineering_Lukla_Phakding.kml
(168, 3)
(90, 3)
remove unrealistic altitude change shape=(168, 3) --> (90, 3)


/Users/yumikondo/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
No geometries found
Object does not have a geometry


  distance = 6.9km
  Elevation gain 358m, Elevation loss -583m
  N samples =  90
____________
Move_2018_04_25_08_12_00_Mountaineering_Phakding_NamcheBzaar.kml
(280, 3)
(153, 3)
remove unrealistic altitude change shape=(280, 3) --> (153, 3)
  distance = 10.1km
  Elevation gain 1259m, Elevation loss -479m
  N samples = 153
____________
Move_2018_04_26_07_58_23_Trekking_NamcheBzaar_NamcheBzaar.kml


No geometries found
Object does not have a geometry


...

  distance = 7.1km
  Elevation gain 103m, Elevation loss -464m
  N samples = 153
____________
Move_2018_05_05_07_52_34_Mountaineering_Machamo_Dhole_part1.kml
(115, 3)
(115, 3)
remove unrealistic altitude change shape=(115, 3) --> (115, 3)


No geometries found
Object does not have a geometry


  distance = 4.8km
  Elevation gain 101m, Elevation loss -472m
  N samples = 115
____________
Move_2018_05_05_10_02_35_Mountaineering_Dhole_NamcheBzaar.kml
(288, 3)
(286, 3)
remove unrealistic altitude change shape=(288, 3) --> (286, 3)


No geometries found
Object does not have a geometry


  distance = 11.4km
  Elevation gain 499m, Elevation loss -1096m
  N samples = 286
____________
Move_2018_05_07_07_34_55_Mountaineering_NamcheBzaar_Lukla.kml
(266, 3)
(248, 3)
remove unrealistic altitude change shape=(266, 3) --> (248, 3)
  distance = 16.7km
  Elevation gain 578m, Elevation loss -1186m
  N samples = 248


In [206]:
dfs["date"] = pd.to_datetime(dfs["date"],format='%Y%m%d')

day0 = np.datetime64('2018-04-23')
unidate = np.unique(dfs["date"])
dfs = dfs.fillna(0)
cs,days = [], []
for ii , date in enumerate(unidate,1):
    dfdate = dfs["dist"].loc[date == dfs["date"].values].values
    cstemp = np.cumsum(dfdate)
    cstemp[0] = 0
    cs.extend(list(cstemp))
    Day = (date - np.datetime64('2018-04-23'))/np.timedelta64(1,'D')
    days.extend([int(Day)]*len(cstemp))
dfs["csdist"] = cs
dfs["Day"] = days

In [197]:
date

numpy.datetime64('2018-05-07T00:00:00.000000000')

14.0

In [207]:
dfs.head(5)

,lon,lat,altitude,date,move,dist,gain,loss,csdist,Day
3,86.731856,27.688181,2860.899902,2018-04-24,Mountaineering,0.000000,358.099853,-582.699706,0.000000,1
4,86.731743,27.688181,2858.500000,2018-04-24,Mountaineering,0.011130,358.099853,-582.699706,0.011130,1
5,86.731222,27.688300,2848.600098,2018-04-24,Mountaineering,0.052994,358.099853,-582.699706,0.064124,1
8,86.729822,27.689310,2828.100098,2018-04-24,Mountaineering,0.177860,358.099853,-582.699706,0.241984,1
9,86.729306,27.689903,2823.500000,2018-04-24,Mountaineering,0.083267,358.099853,-582.699706,0.325252,1


In [208]:
dfs.tail(5)

,lon,lat,altitude,date,move,dist,gain,loss,csdist,Day
261,86.729075,27.690058,2872.000000,2018-05-07,Mountaineering,0.075092,578.389538,-1186.321137,16.474535,14
262,86.729570,27.689688,2871.000000,2018-05-07,Mountaineering,0.063801,578.389538,-1186.321137,16.538336,14
263,86.729966,27.689244,2871.254200,2018-05-07,Mountaineering,0.062930,578.389538,-1186.321137,16.601266,14
264,86.730447,27.688892,2870.000000,2018-05-07,Mountaineering,0.061460,578.389538,-1186.321137,16.662726,14
265,86.730907,27.688873,2869.068401,2018-05-07,Mountaineering,0.045356,578.389538,-1186.321137,16.708082,14


In [209]:
dfs.to_csv(dir_move + "trekking.csv")